# Clustering - 군집 분석

---

In [ ]:
# Visual Python: Data Analysis > Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# Visual Python: Visualization > Chart Style
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.rc('figure', figsize=(8, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 유럽 국가별 단백질 섭취원 비율

#### 데이터 로드

In [ ]:
# Visual Python: Data Analysis > File
df = pd.read_csv('./data/protein.csv', index_col='Country')
df

### 1.1 K-Means 알고리즘

In [ ]:
# Visual Python: Machine Learning > Clustering
from sklearn.cluster import KMeans

model = KMeans(n_clusters=5)

In [ ]:
# Visual Python: Machine Learning > Fit/Predict
pred = model.fit_predict(df)
pred

In [ ]:
# Visual Python: Data Analysis > Frame
df['gpnum'] = pred
df

### 1.2 병합군집(Agglomerative Clustering)

In [ ]:
# Visual Python: Machine Learning > Clustering
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=5)

In [ ]:
# Visual Python: Machine Learning > Fit/Predict
pred = model.fit_predict(df)
pred

### 1.3 GMM(Gaussian Mixture Model)

In [ ]:
# Visual Python: Machine Learning > Clustering
from sklearn.mixture import GaussianMixture

model = GaussianMixture(n_components=5)

In [ ]:
# Visual Python: Machine Learning > Fit/Predict
pred = model.fit_predict(df)
pred

### 1.4 DBSCAN

In [ ]:
# Visual Python: Machine Learning > Clustering
from sklearn.cluster import DBSCAN

model = DBSCAN()

In [ ]:
# Visual Python: Machine Learning > Fit/Predict
pred = model.fit_predict(df)
pred

# 2 K-Means
- 거리 기반 군집 알고리즘

### 2.1 K-Means 알고리즘

#### 2.1.1 데이터 생성

In [ ]:
# Visual Python: Machine Learning > Data Sets
from sklearn.datasets import make_blobs

_X, _y = make_blobs(n_features=2)
# Create DataFrame
_feature_names = ['X{}'.format(i+1) for i in range(len(_X[0]))]
df_bl = pd.DataFrame(data=_X, columns=_feature_names)
df_bl['target'] = _y
df_bl

#### 2.1.2 K-Means 군집 모델 생성

In [ ]:
# Visual Python: Machine Learning > Clustering
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)

In [ ]:
# Visual Python: Machine Learning > Fit/Predict
kmeans.fit(df_bl[['X1', 'X2']])

In [ ]:
# Visual Python: Machine Learning > Fit/Predict
pred = kmeans.predict(df_bl[['X1', 'X2']])
pred

In [ ]:
import matplotlib as mpl
from matplotlib.colors import ListedColormap, colorConverter, LinearSegmentedColormap

cm_cycle = ListedColormap(['#0000aa', '#ff5050', '#50ff50', '#9040a0', '#fff000'])
cm3 = ListedColormap(['#0000aa', '#ff2020', '#50ff50'])
cm2 = ListedColormap(['#0000aa', '#ff2020'])

# create a smooth transition from the first to to the second color of cm3
# similar to RdBu but with our red and blue, also not going through white,
# which is really bad for greyscale

cdict = {'red': [(0.0, 0.0, cm2(0)[0]),
                 (1.0, cm2(1)[0], 1.0)],

         'green': [(0.0, 0.0, cm2(0)[1]),
                   (1.0, cm2(1)[1], 1.0)],

         'blue': [(0.0, 0.0, cm2(0)[2]),
                  (1.0, cm2(1)[2], 1.0)]}

ReBl = LinearSegmentedColormap("ReBl", cdict)

def discrete_scatter(x1, x2, y=None, markers=None, s=10, ax=None,
                     labels=None, padding=.2, alpha=1, c=None, markeredgewidth=None):
    if ax is None:
        ax = plt.gca()

    if y is None:
        y = np.zeros(len(x1))

    unique_y = np.unique(y)

    if markers is None:
        markers = ['o', '^', 'v', 'D', 's', '*', 'p', 'h', 'H', '8', '<', '>'] * 10

    if len(markers) == 1:
        markers = markers * len(unique_y)

    if labels is None:
        labels = unique_y

    # lines in the matplotlib sense, not actual lines
    lines = []

    current_cycler = mpl.rcParams['axes.prop_cycle']

    for i, (yy, cycle) in enumerate(zip(unique_y, current_cycler())):
        mask = y == yy
        # if c is none, use color cycle
        if c is None:
            color = cycle['color']
        elif len(c) > 1:
            color = c[i]
        else:
            color = c
        # use light edge for dark markers
        if np.mean(colorConverter.to_rgb(color)) < .4:
            markeredgecolor = "grey"
        else:
            markeredgecolor = "black"

        lines.append(ax.plot(x1[mask], x2[mask], markers[i], markersize=s,
                             label=labels[i], alpha=alpha, c=color,
                             markeredgewidth=markeredgewidth,
                             markeredgecolor=markeredgecolor)[0])

    if padding != 0:
        pad1 = x1.std() * padding
        pad2 = x2.std() * padding
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.set_xlim(min(x1.min() - pad1, xlim[0]), max(x1.max() + pad1, xlim[1]))
        ax.set_ylim(min(x2.min() - pad2, ylim[0]), max(x2.max() + pad2, ylim[1]))

    return lines

In [ ]:
# 클러스터 할당 시각화
discrete_scatter(df_bl['X1'], df_bl['X2'], pred, markers='o')

plt.title('KMeans Clustering')
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

# 3 병합군집(Agglomerative Clustering)

#### linkage 옵션
- ward: 기본값인 ward 연결은 모든 클러스터 내의 분산을 가장 작게 증가시키는 두 클러스터를 병합. 크기가 비교적 비슷한 클러스터 생성
- average: 클러스터 포인트 사이의 평균 거리가 가장 짧은 두 클러스터를 병합
- complete: 클러스터 포인트 사이의 최대 거리가 가장 짧은 두 클러스터를 병합

### 3.1 병합군집(Agglomerative Clustering)

In [ ]:
# 데이터 생성
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=0, n_samples=12)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, ward

# 덴드로 그램
linkage_array = ward(X)
dendrogram(linkage_array)

# 두 개와 세 개의 클러스터를 구분하는 커트라인을 표시
ax = plt.gca()
bounds = ax.get_xbound()
ax.plot(bounds, [7.25, 7.25], '--', c='k')
ax.plot(bounds, [4, 4], '--', c='k')

ax.text(bounds[1], 7.25, ' 두 개 클러스터', va='center', fontdict={'size': 15})
ax.text(bounds[1], 4, ' 세 개 클러스터', va='center', fontdict={'size': 15})

plt.xlabel('샘플 번호')
plt.ylabel('클러스터 거리')
plt.show()

# 4 GMM(Gaussian Mixture Model)

- 확률 기반 군집 알고리즘

In [ ]:
# 무작위로 클러스터 데이터 생성
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)

# 데이터가 길게 늘어지도록 변경
transformation = rng.normal(size=(2, 2))
X = np.dot(X, transformation)

In [ ]:
# 세개의 클러스터로 GMM 알고리즘 적용
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3, random_state=0)
gmm_label = gmm.fit(X).predict(X)

In [ ]:
# 클러스터 할당 시각화
discrete_scatter(X[:,0], X[:,1], gmm_label, markers='o')

plt.title('원형이 아닌 클러스터에 GMM 적용')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

# 5 DBSCAN

#### 옵션
- 입실론 주변 영역(epsilon): 개별 데이터를 중심으로 입실론 반경을 가지는 원형의 영역
- 최소 데이터 개수(min points): 개별 데이터의 입실론 주변 영역에 포함되는 타 데이터의 개수

#### 포인트 구분
- 핵심 포인트(Core Point): 주변 영역 내에 최소 데이터 개수 이상의 타 데이터를 가지고 있는 데이터 포인트
- 이웃 포인트(Neighbor Point): 주변 영역 내에 위치한 타 데이터 포인트
- 경계 포인트(Border Point): 핵심 포인트를 이웃 포인트로 가지고 있는 데이터 포인트
- 잡음 포인트(Noise Point): 최소 데이터 개수 이상의 이웃 포인트를 가지고 있지 않고 핵심 포인트를 이웃 포인트로도 가지고 있지 않은 데이터 포인트

### 5.1 DBSCAN 적용

In [ ]:
# 데이터 생성
from sklearn.datasets import make_blobs
X, y = make_blobs(random_state=0, n_samples=12)

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
clusters = dbscan.fit_predict(X)

print('클러스터 레이블:\n', clusters)

### 5.2 DBSCAN 적용 - 데이터 스케일링

In [ ]:
# 데이터 생성
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

In [ ]:
# 스케일 조정
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
clusters = dbscan.fit_predict(X_scaled)

In [ ]:
# 클러스터 할당 시각화

plt.scatter(X_scaled[:,0], X_scaled[:,1], c=clusters, cmap=cm2, s=60, edgecolors='black')

plt.title('기본값 eps=0.5를 사용해 DBSCAN으로 찾은 클러스터 할당')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

### 5.3 DBSCAN 적용 - 원형 데이터

In [ ]:
# 데이터 생성
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=1000, shuffle=True, noise=0.05, random_state=0, factor=0.5)

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.2, min_samples=10, metric='euclidean')
clusters = dbscan.fit_predict(X)

In [ ]:
# 클러스터 할당 시각화

plt.scatter(X[:,0], X[:,1], c=clusters, cmap=cm2, s=60, edgecolors='black')

plt.title('원형 데이터 분포를 DBSCAN으로 찾은 클러스터 할당')
plt.xlabel('특성 0')
plt.ylabel('특성 1')
plt.show()

---

In [ ]:
# End of file